In [10]:
import numpy as np

# Reynolds And Mach and AoA

In [3]:
def ms2mach(ms):
    return ms/340.29
def Re(v,c,n):
    return (v*c)/n

In [4]:
chordMax = 0.18
chordMin = 0.11
umax = 30
umin = 5
ne = 1.56e-5

In [22]:
Machmin = ms2mach(10)
Machmax = ms2mach(30)
Remax = Re(umax,chordMax,ne)
Remin = Re(umin,chordMin,ne)
AoAmax = 15
AoAmin = -6
NoAoA = (AoAmax - AoAmin)*2 + 1

In [27]:
angles = np.linspace(AoAmin,AoAmax,NoAoA)
Reynolds = np.logspace(np.log10(Remin),np.log10(Remax),20,base=10)
Mach = np.linspace(Machmax,Machmin,10)

In [28]:
Reynolds

array([ 35256.41025641,  39760.35494496,  44839.67068262,  50567.86011366,
        57027.81571645,  64313.01933835,  72528.8949691 ,  81794.33432854,
        92243.41734558, 104027.35242534, 117316.66458197, 132303.66310164,
       149205.22444519, 168265.93066161, 189761.60872851, 214003.32203701,
       241341.87178186, 272172.87339632, 306942.48149268, 346153.84615385])

In [29]:
Mach

array([0.0881601 , 0.08162972, 0.07509934, 0.06856897, 0.06203859,
       0.05550821, 0.04897783, 0.04244745, 0.03591708, 0.0293867 ])

In [30]:
angles

array([-6. , -5.5, -5. , -4.5, -4. , -3.5, -3. , -2.5, -2. , -1.5, -1. ,
       -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,
        5. ,  5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. ,
       10.5, 11. , 11.5, 12. , 12.5, 13. , 13.5, 14. , 14.5, 15. ])